# Things left to do on model front

    1) Either use original polyglot embeddings or original code on word2vec to make sure our model is getting similar results; would be really dumb if we have a small mistake in our model and aren't actually training it correctly

    2) Build 1-2 more models of increasing complexity.  Perhaps we add attention to one of the biLSTM layers or attempt encoder decoder approach.
    
    3) Build script that takes a feature and works through getting avg. embeddings

In [1]:
# Useful library for embeddings

import class_lstm
from class_lstm import MimicLSTM

import word2vec_preprocess
import word2vec_preprocess as preprocess

import gensim
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import importlib

from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing import sequence 
import tensorflow as tf

import gensim

word2vec = gensim.models.KeyedVectors.load_word2vec_format(
    './word2vec_model/GoogleNews-vectors-negative300.bin', binary=True) 

data_dict, chardict = preprocess.run()

/home/cgleach/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def nn_word(word, word_emb_dict, lstm_model, char_dict, word2vec):
    indices = np.array([char_dict[letter] for letter in word])
    indices = sequence.pad_sequences([indices], maxlen=25) 
    prediction = lstm_model.predict(tf.reshape(indices,(1,-1)), steps=1)
    
    if word in word2vec:
        actual = np.reshape(word2vec[word],(1,-1))
        actual_sim = cosine_similarity(actual,prediction)
        print("Actual sim is %s" %actual_sim)
    else:
        actual_sim = "Not Known"
    
    best_word = ''
    best_sim = 0
    total_sim = 0
    total_count = 0
    
    for k,v in word_emb_dict.items():
        
        v = np.reshape(v,(1,-1))
        sim = cosine_similarity(prediction, v)
        total_sim += sim
        total_count += 1
        
        if sim > best_sim:
            best_word = k
            best_sim = sim
            print(best_word, best_sim)
    avg_sim = total_sim/total_count
    
    print("actual sim was %s while avg was %s" %(actual_sim,avg_sim))
    return actual_sim, avg_sim

In [3]:
# Reload our packages as we test 
importlib.reload(class_lstm)
importlib.reload(word2vec_preprocess)

<module 'word2vec_preprocess' from '/home/cgleach/w266/final_project/w266-final-project-OOV-fake-bias-detection/original_work/word2vec_preprocess.py'>

In [ ]:
# Original model as outlined in paper
layers = 1
H = 50
character_dim = 20
output_dim = 300

model = MimicLSTM(layers, H, chardict, character_dim, data_dict, 
                  batch_size=1000, epochs=30, full_set=True)

model.save_model("./mimic_model_original.h5")

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 20)            1120      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               28400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               30300     
Total params: 59,820
Trainable params: 59,820
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
760310/760310 [==============================] - 121s 159us/step - loss: 0.0193
Epoch 2/30
760310/760310 [==============================] -

In [ ]:
loadmodel = MimicLSTM(layers, H, chardict, character_dim, data_dict, 
                  train=False, load_path="./mimic_model_original.h5")

In [ ]:
# Double the layers
layers = 2
H = 50
character_dim = 20
output_dim = 300

model_complex = MimicLSTM(layers, H, chardict, character_dim, data_dict, batch_size=1000, 
                          epochs = 30, full_set = True)

model.save_model("./mimic_model_complex.h5")

In [5]:
# Super complicated model

layers = 2
H = 100
character_dim = 100
output_dim = 300
recurrent_dropout = .05
dense_dropout = .05
attention = True

model_complex_attention = MimicLSTM(layers, H, chardict, character_dim, data_dict, batch_size=1000, 
                          epochs = 30, full_set = True, use_attention=attention, 
                        recurrent_dropout = recurrent_dropout, dense_dropout = dense_dropout)

#model_complex_attention.save_model("./mimic_model_complex_attention.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 100)           5600      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 200)           160800    
_________________________________________________________________
seq_self_attention_1 (SeqSel (None, 25, 200)           12865     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               240800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)      

In [6]:
layers = 2
H = 100
character_dim = 100
output_dim = 300
recurrent_dropout = .05
dense_dropout = .05
attention = True

model_complex_attention_cosine = MimicLSTM(layers, H, chardict, character_dim, data_dict, batch_size=1000, 
                          epochs = 30, full_set = True, use_attention=attention, 
                        recurrent_dropout = recurrent_dropout, dense_dropout = dense_dropout)

model_complex_attention_cosine.save_model("./mimic_model_complex_attention_cosine_loss.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 100)           5600      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 25, 200)           160800    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               240800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               60300     
Total params: 467,500
Trainable params: 467,500
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
 19000/760310 [..............................] - ETA: 18:06 - loss: 0.7247

KeyboardInterrupt: 